# Algoritmo Naive Bayes (Mod. de Clasificación)

En este proyecto practicaré con un conjunto de datos para crear un clasificador de reseñas de la tienda de Google Play

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
# split para modelado
from sklearn.model_selection import train_test_split
# Scaled | Escalado
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Encoding | Codificación
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
# To save models
import json
import pickle
# Feature Selection
from sklearn.feature_selection import f_classif, SelectKBest
# Modelado
from sklearn.linear_model import LogisticRegression
# Métricas
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, mean_absolute_percentage_error, root_mean_squared_error, r2_score
# Optimizar
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# from tqdm import tqdm
import warnings
from utils import get_regression_metrics
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV, ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from pickle import dump
from sklearn.tree import DecisionTreeRegressor
from utils import get_classifier_metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



In [2]:
# Mostrar todas las filas
pd.set_option("display.max_rows", None)

# Mostrar todas las columnas
pd.set_option("display.max_columns", None)

# Ajustar el ancho de cada columna si es necesario
pd.set_option("display.max_colwidth", None)

# Aumentar el ancho total del display
pd.set_option("display.width", 0)

In [3]:
df_ini = pd.read_csv('../data/raw/playstore_reviews.csv', sep=',')
df_ini.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offline. i mean for some people like me it's a big pressure to be seen online like you need to response on every message or else you be called seenzone only. if only i wanna do on facebook is to read on my newsfeed and just wanna response on message i want to. pls reconsidered my review. i tried to turn off chat but still can see me as online.,0
1,com.facebook.katana,"messenger issues ever since the last update, initial received messages don't get pushed to the messenger app and you don't get notification in the facebook app or messenger app. you open the facebook app and happen to see you have a message. you have to click the icon and it opens messenger. subsequent messages go through messenger app, unless you close the chat head... then you start over with no notification and having to go through the facebook app.",0
2,com.facebook.katana,profile any time my wife or anybody has more than one post and i view them it would take me to there profile so that i can view them all at once. now when i try to view them it tells me that the page that i requested is not available. i've restarted my phone i even cleard the cache and i've uninstalled and reinstalled the app and it is still doing it. please fix it thank you,0
3,com.facebook.katana,the new features suck for those of us who don't have a working back button can you guys make the videos able to be slid to the left to exit the video. as i have to force close facebook to exit,0
4,com.facebook.katana,"forced reload on uploading pic on replying comment last night i tried to reply a comment by uploading a photo from my phone. when i press on the button to select photos the app automatically goes back to the main page. on other occasions, i could enter to my gallery to select my image but as soon as i selected an image and press done, the program, again, forced reload and suddenly go back to the main page. please fix this and i will change my rating. thank you.",0


In [4]:
df_ini.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [5]:
df= df_ini.drop_duplicates()
df.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offline. i mean for some people like me it's a big pressure to be seen online like you need to response on every message or else you be called seenzone only. if only i wanna do on facebook is to read on my newsfeed and just wanna response on message i want to. pls reconsidered my review. i tried to turn off chat but still can see me as online.,0
1,com.facebook.katana,"messenger issues ever since the last update, initial received messages don't get pushed to the messenger app and you don't get notification in the facebook app or messenger app. you open the facebook app and happen to see you have a message. you have to click the icon and it opens messenger. subsequent messages go through messenger app, unless you close the chat head... then you start over with no notification and having to go through the facebook app.",0
2,com.facebook.katana,profile any time my wife or anybody has more than one post and i view them it would take me to there profile so that i can view them all at once. now when i try to view them it tells me that the page that i requested is not available. i've restarted my phone i even cleard the cache and i've uninstalled and reinstalled the app and it is still doing it. please fix it thank you,0
3,com.facebook.katana,the new features suck for those of us who don't have a working back button can you guys make the videos able to be slid to the left to exit the video. as i have to force close facebook to exit,0
4,com.facebook.katana,"forced reload on uploading pic on replying comment last night i tried to reply a comment by uploading a photo from my phone. when i press on the button to select photos the app automatically goes back to the main page. on other occasions, i could enter to my gallery to select my image but as soon as i selected an image and press done, the program, again, forced reload and suddenly go back to the main page. please fix this and i will change my rating. thank you.",0


In [6]:
df_ini.info()

#Veo que no teniamos duplicados en el data.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [7]:
# elimino la columna con la cual no vamos a trabajar.

df_ini.drop('package_name', axis=1, inplace=True)


In [8]:
# 1º Eliminar espacios y convertir a minúsculas el texto.

df["review"] = df["review"].str.strip().str.lower()

## Split

In [9]:
# 2º Dividimos el conjunto de datos en muestras de train y test

X = df_ini.drop('polarity', axis=1)
y = df_ini['polarity']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=18)

In [10]:
# 3º Transformar el texto en una matriz de recuento de palabras. Esta es una forma de obtener características numéricas a partir del texto.
#  Para ello, utilizamos el conjunto de train para entrenar el transformador y la aplicamos en test.

vec_model = CountVectorizer(stop_words = "english")
X_train = vec_model.fit_transform(X_train["review"]).toarray()
X_test = vec_model.transform(X_test["review"]).toarray()

## Modelo MultinomialNB

In [11]:
model_multi = MultinomialNB()


In [ ]:
# Entrenamiento
model_multi.fit(X_train, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [13]:
# predicción
y_pred_test = model_multi.predict(X_test)
y_pred_train = model_multi.predict(X_train)


In [14]:
#Métricas 
accuracy_train = (accuracy_score(y_train, y_pred_train))

accuracy_test = (accuracy_score(y_test, y_pred_test))

accuracy = pd.DataFrame(data=[accuracy_train, accuracy_test],
                        columns=['MultinomialNB'],
                        index=['Train set', 'Test set'])

accuracy

,MultinomialNB
Train set,0.955056
Test set,0.815642


## Modelo GaussianNB

In [15]:
model_gau = GaussianNB()

In [16]:
#Entreno modelo
model_gau.fit(X_train, y_train)

# predicción
y_pred_test_gau = model_gau.predict(X_test)
y_pred_train_gau = model_gau.predict(X_train)

In [17]:
#Métricas 
accuracy_train_gau = (accuracy_score(y_train, y_pred_train_gau))

accuracy_test_gau = (accuracy_score(y_test, y_pred_test_gau))

accuracy_gau = pd.DataFrame(data=[accuracy_train_gau, accuracy_test_gau],
                        columns=['GaussianNB'],
                        index=['Train set', 'Test set'])

accuracy_gau

,GaussianNB
Train set,0.984551
Test set,0.787709


## Modelo BernoulliNB

In [18]:
model_ber = BernoulliNB()

In [19]:
#Entreno modelo
model_ber.fit(X_train, y_train)

# predicción
y_pred_test_ber = model_ber.predict(X_test)
y_pred_train_ber = model_ber.predict(X_train)

In [20]:
#Métricas 
accuracy_train_ber = (accuracy_score(y_train, y_pred_train_ber))

accuracy_test_ber = (accuracy_score(y_test, y_pred_test_ber))

accuracy_ber = pd.DataFrame(data=[accuracy_train_ber, accuracy_test_ber],
                        columns=['BernoulliNB'],
                        index=['Train set', 'Test set'])

accuracy_ber

,BernoulliNB
Train set,0.907303
Test set,0.715084


 ### 1º Conclusión:
> De los 3 modelos probados el que mejor resultado inicial da es el modelo MultinomialNB.
>
> Voy a Hiperparametrizar para ver si mejoro los resultados.

### Hiperparametrización de MultinomialNB

In [21]:
# Definir hiperparámetros a probar
hiperparam = {"alpha": np.linspace(0.01, 10.0, 200),"force_alpha":[True, False], "fit_prior": [True, False]}

# We initialize the random search
random_search = RandomizedSearchCV(model_multi, hiperparam,
                                   n_iter=50,
                                   scoring="accuracy",
                                   cv=5,
                                   random_state=18)
random_search

,estimator,MultinomialNB()
,param_distributions,"{'alpha': array([ 0.01 ... 10. ]), 'fit_prior': [True, False], 'force_alpha': [True, False]}"
,n_iter,50
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,18
,error_score,nan


In [22]:
# Definir hiperparámetros a probar
hiperparam = {"alpha": np.linspace(0.01, 10.0, 200),"force_alpha":[True, False], "fit_prior": [True, False]}

# Implementar GridSearchCV
grid_search = GridSearchCV(MultinomialNB(),
                                hiperparam,
                                cv=4,
                                n_jobs=-1,
                                verbose=1)

# Entrenamos
grid_search.fit(X_train, y_train)

Fitting 4 folds for each of 800 candidates, totalling 3200 fits


,estimator,MultinomialNB()
,param_grid,"{'alpha': array([ 0.01 ... 10. ]), 'fit_prior': [True, False], 'force_alpha': [True, False]}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,4
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,np.float64(1.1646231155778894)


### Best Estimator

In [23]:
# Mejor modelo
grid_search.best_estimator_

,alpha,np.float64(1.1646231155778894)
,force_alpha,True
,fit_prior,False
,class_prior,None


In [24]:
# Entrenamos
grid_search.best_estimator_.fit(X_train, y_train)

,alpha,np.float64(1.1646231155778894)
,force_alpha,True
,fit_prior,False
,class_prior,None


In [25]:
y_pred_test_grid = grid_search.best_estimator_.predict(X_test)
y_pred_train_grid = grid_search.best_estimator_.predict(X_train)


In [26]:
get_classifier_metrics(y_pred_test_grid, y_test, y_pred_train_grid, y_train)

,Accuracy,F1 Score,Precision,Recall
Train set,0.956461,0.956461,0.956461,0.956461
Test set,0.826816,0.826816,0.826816,0.826816


In [27]:
with open('../models/Playstore-NaiveBayes.pkl', 'wb') as file:
    pickle.dump(model_multi, file)


> ### Conslusión:
> - Aunque hay poco para hipermetrizar, hemos conseguido en mejorar la eficacia del modelo un 1% de diferencia entre test y train.
> - Pasamos de tener un test inicial de 81,5% a un 82,6%.

## Random Forest

In [28]:
model_forest_clas = RandomForestClassifier(random_state = 20)
model_forest_clas.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [29]:
y_pred_test_random = model_forest_clas.predict(X_test)
y_pred_train_random = model_forest_clas.predict(X_train)

In [30]:
get_classifier_metrics(y_pred_test_random, y_test, y_pred_train_random, y_train)

,Accuracy,F1 Score,Precision,Recall
Train set,1.000000,1.000000,1.000000,1.000000
Test set,0.798883,0.798883,0.798883,0.798883


In [31]:
# Definir hiperparámetros de random forest
hiperparam_random = {'n_estimators': [15, 20, 30],
                   'max_depth': [10, 15, 20, 25, 30],
                   'min_samples_split': [2, 5, 6, 7]}

# Implementar GridSearchCV
grid_search_random = GridSearchCV(RandomForestClassifier(random_state=18),
                                hiperparam_random,
                                cv=4,
                                n_jobs=-1,
                                verbose=1)

# Entrenamos
grid_search_random.fit(X_train, y_train)

Fitting 4 folds for each of 60 candidates, totalling 240 fits


,estimator,RandomForestC...ndom_state=18)
,param_grid,"{'max_depth': [10, 15, ...], 'min_samples_split': [2, 5, ...], 'n_estimators': [15, 20, ...]}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,4
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,15


In [32]:
# Mejor modelo
grid_search_random.best_estimator_

,n_estimators,15
,criterion,'gini'
,max_depth,30
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [33]:
# Entrenamos
grid_search_random.best_estimator_.fit(X_train, y_train)

,n_estimators,15
,criterion,'gini'
,max_depth,30
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [34]:
y_pred_test_grid = grid_search_random.best_estimator_.predict(X_test)
y_pred_train_grid = grid_search_random.best_estimator_.predict(X_train)


In [35]:
#Random Forest - métricas.
get_classifier_metrics(y_pred_test_grid, y_test, y_pred_train_grid, y_train)

,Accuracy,F1 Score,Precision,Recall
Train set,0.918539,0.918539,0.918539,0.918539
Test set,0.770950,0.770950,0.770950,0.770950


## Conclusión Random Forest.
> Aunque hiperparametrizando hemos arreglado el sobreentreno. (overfiting) el resultado de test por debajo del 70% no es un buen resultado.